# Imports

In [1]:
import LightPipes as lp
import numpy as np
import matplotlib.pyplot as plt

# Functions

In [2]:
def plot_Field(Field: lp.field.Field, cmap="jet", title="Field") -> None:
    """
        Given an intensity array I and the extent (in mm) of the image, plot it and its intensity through the middle.
    """
    # Get the intensity
    I = lp.Intensity(Field, flag=0)

    # Get the size
    size = Field.siz

    # Array size
    N = Field.N

    # Setup subplots
    cols = 2
    rows = 1
    plt.figure(figsize=(6 * cols, 5 * rows))

    # Plot the field image
    plt.subplot(rows, cols, 1)
    plt.imshow(I, extent=(0, size / lp.mm, 0, size / lp.mm), cmap=cmap)
    plt.colorbar()
    plt.title(title + f" N={N}")
    plt.xlabel('Position (mm)')
    plt.ylabel('Position (mm)')

    # Plot the field Intensity
    plt.subplot(rows, cols, 2)
    xx = (np.arange(N)/N * size - (size / 2))

    plt.plot(xx * (1/lp.mm), I[N//2], label="Simulation")
    plt.title("Intensity profile through the center")
    plt.xlabel("x position (mm)")
    plt.ylabel("Intensity (a.u.)")
    plt.grid("on")

# Gaussian Beam and Second Pass Through a Lens

In [5]:
# Parameters
size = 40 * lp.mm # size of the grid
wave = 632.8 * lp.nm # wavelength of HeNe laser
N = 2048 # number (NxN) of grid pixels
h = size / N

k = 2 * np.pi / wave

# Laser Beam
w0 = 5 * lp.mm # laser beam waist

# Parameters
d1 = 10 * lp.m # distance from the lens to 0
f = 10 * lp.m # focal length of the lens
d2 = 10 * lp.m # distance from the lens to the detection plane


# Create the beam
Field = lp.Begin(size, wave, N)
Field = lp.GaussBeam(Field, w0)
plot_Field(Field, title="Field at the start")

# Propagate the laser some distance
Field = lp.Forvard(Field, d1)
plot_Field(Field, title="Field just before the lens")


# Pass the beam through the lens and propagate up to the focus
Field = lp.Lens(Field, f)
Field = lp.Forvard(Field, f)
plot_Field(Field, title="Field after lens and some propagation")


# "Reflect" the beam and propagate back to the lens
Field = lp.Forvard(Field, f)
plot_Field(Field, title="Field just before the lens")


# Propagate to the detection plane
Field = lp.Lens(Field, f)
Field = lp.Forvard(Field, d1)
plot_Field(Field, title="Field at the detection plane")


# Predict the beam using the beam parameter
I = lp.Intensity(Field, flag=0)
Imax = np.max(I[N//2])

# ABCD matriz of the propagtion -> lens -> propagation -> lens -> propagation
prop1 = np.array([[1, d1], [0, 1]])
prop2 = np.array([[1, f], [0, 1]])
lens = np.array([[1, 0], [-1/f, 1]])

ABCD = prop1 @ (lens @ (prop2 @ (prop2 @ (lens @ prop1))))


# Beam parameter calculated using ABCD matrices
q0 = 1j * np.pi * w0**2 / wave
qz = (ABCD[0, 0] * q0 + ABCD[0, 1]) / (ABCD[1, 0] * q0 + ABCD[1, 1])

# Beam parameter through the lens
xx = np.arange(N)/N * size - (size / 2)
yy = Imax * np.abs(np.exp((-1j * k * xx**2) / (2 * qz)))**2

plt.plot(xx / lp.mm, yy, label="Prediction")
plt.legend()
plt.show()


# Compare with starting field
Field = lp.Begin(size, wave, N)
Field = lp.GaussBeam(Field, w0)
plot_Field(Field, title="Field at the start")
plt.plot(xx / lp.mm, yy, label="Prediction")
plt.legend()
plt.show()